# 1. Connect to Azure Machine Learning Workspace

## 1.1 Import the required libraries

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace and the registry

In [ ]:
# Get a handle to workspace
ml_client_ws = MLClient(
    credential, "<SUBSCRIPTION_ID>", "<RESOURCE_GROUP>", "<AML_WORKSPACE_NAME>"
)
ml_client_registry = MLClient(
    credential, registry_name="azureml"
)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster-D13-v2"

# Note above compute has an MSI attached to it, which is necessary to get credentials and run cli/sdk v2 on that compute

# 2. Load registered components to create pipeline

## 2.1 Use worskpace handle to load the components from workspace

In [ ]:
model_downloader = ml_client_registry.components.get(name="download_model", label="latest")
mlflow_converter = ml_client_registry.components.get(name="mlflow_converter", label="latest")
model_registration = ml_client_registry.components.get(name="register_model", label="latest")
model_deployment = ml_client_registry.components.get(name="deploy_model", label="latest")

# 3. Build pipeline using above components

## 3.1 Connect components and build pipeline framework

In [ ]:
@pipeline(
    description="Model Publishing Pipeline",
)
def model_publishing_pipeline():

    model_downloading_node = model_downloader(
        model_uri_type="git",
        model_uri="https://huggingface.co/ai4bharat/indic-bert",
        model_type="custom_model",
    )

    mlflow_converter_node = mlflow_converter(
        model_path=model_downloading_node.outputs.model_output,
        hf_task_type="text-classification",
        model_flavor="hftransformers",
    )

    model_registration_node = model_registration(
        model_path=mlflow_converter_node.outputs.mlflow_model_output,
        name_for_registered_model="ai4bharat-indic-bert",
    )
    # Note: do pass registry name if you want to register it in registry

    model_deployment_node = model_deployment(
        registered_model_id_path=model_registration_node.outputs.registered_model_details_path,
        endpoint_name="indic-bert-ep",
        deployment_name="ai4bharat-indic-bert",
    )

## 3.2 Pass parameters to pipeline and set up compute

In [ ]:
pipeline_job = model_publishing_pipeline()

# set pipeline level compute
pipeline_job.settings.default_compute = cluster_name

## 3.3 Submit job to workspace

In [ ]:
pipeline_job = ml_client_ws.jobs.create_or_update(
    pipeline_job, experiment_name="Pipeline with registered component in prod registry"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client_ws.jobs.stream(pipeline_job.name)